In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mikhail1681/walmart-sales")

print("Path to dataset files:", path)

c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 122k/122k [00:00<00:00, 322kB/s]

Extracting files...
Path to dataset files: C:\Users\USER\.cache\kagglehub\datasets\mikhail1681\walmart-sales\versions\2


In [1]:
import pandas as pd
walmart = pd.read_csv("Walmart_Sales.csv")      # reading csv file 
print(walmart.head())

   Store        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0      1  05-02-2010    1643690.90             0        42.31       2.572   
1      1  12-02-2010    1641957.44             1        38.51       2.548   
2      1  19-02-2010    1611968.17             0        39.93       2.514   
3      1  26-02-2010    1409727.59             0        46.63       2.561   
4      1  05-03-2010    1554806.68             0        46.50       2.625   

          CPI  Unemployment  
0  211.096358         8.106  
1  211.242170         8.106  
2  211.289143         8.106  
3  211.319643         8.106  
4  211.350143         8.106  


In [2]:
# to check the shape of the data that is rows and columns
walmart.shape

(6435, 8)

In [4]:
walmart.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [5]:
walmart.dtypes

Store             int64
Date             object
Weekly_Sales    float64
Holiday_Flag      int64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object

### 🧮 A. Sales Performance Analysis

In [3]:
# Which store recorded the highest total sales across all weeks?

shs_week = walmart.groupby("Store", as_index=False)["Weekly_Sales"].sum()
shs = shs_week.sort_values(by="Weekly_Sales", ascending=False)
shs.head()

,Store,Weekly_Sales
19,20,3.013978e+08
3,4,2.995440e+08
13,14,2.889999e+08
12,13,2.865177e+08
1,2,2.753824e+08


In [4]:
# What is the average weekly sales per store?
Avg_weekly_sales = walmart.groupby('Store', as_index=False)['Weekly_Sales'].mean()
Avg_weekly_sales.head()


,Store,Weekly_Sales
0,1,1.555264e+06
1,2,1.925751e+06
2,3,4.027044e+05
3,4,2.094713e+06
4,5,3.180118e+05


In [5]:
# ➤ How consistent is each store’s sales over time?
storeSales_by_time = walmart.groupby(['Store', 'Date'])['Weekly_Sales'].sum().reset_index()
storeSales_by_time.head()

,Store,Date,Weekly_Sales
0,1,01-04-2011,1495064.75
1,1,01-06-2012,1624477.58
2,1,01-07-2011,1488538.09
3,1,01-10-2010,1453329.50
4,1,02-03-2012,1688420.76


## Holiday Impact

In [6]:
walmart.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [7]:
# how do holiday weeks compare with non-holiday weeks?
holiday_weeks = walmart[walmart['Holiday_Flag'] == 1]
non_holiday_weeks = walmart[walmart['Holiday_Flag'] == 0]

holiday_weeks['Weekly_Sales'].sum()
non_holiday_weeks['Weekly_Sales'].sum()

holiday_weeks['Weekly_Sales'].mean()
non_holiday_weeks['Weekly_Sales'].mean()

holiday_weeks['Weekly_Sales'].std()
non_holiday_weeks['Weekly_Sales'].std()

compare_holiday_non_holiday = pd.DataFrame({
    'Holiday': [holiday_weeks['Weekly_Sales'].sum(), holiday_weeks['Weekly_Sales'].mean(), holiday_weeks['Weekly_Sales'].std()],
    'Non-Holiday': [non_holiday_weeks['Weekly_Sales'].sum(), non_holiday_weeks['Weekly_Sales'].mean(), non_holiday_weeks['Weekly_Sales'].std()]
})
compare_holiday_non_holiday.index = ['Sum', 'Mean', 'Standard Deviation']
# print(f"Holiday vs Non-Holiday Sales:\n{compare_holiday_non_holiday}")
print("Holiday vs Non-Holiday Weekly Sales")
compare_holiday_non_holiday

Holiday vs Non-Holiday Weekly Sales


,Holiday,Non-Holiday
Sum,5.052996e+08,6.231919e+09
Mean,1.122888e+06,1.041256e+06
Standard Deviation,6.276849e+05,5.589574e+05


In [6]:
# Do holidays affect all stores equally?
import matplotlib.pyplot as plt
holiday_effect = pd.pivot_table(data = walmart, values = 'Weekly_Sales', columns = 'Holiday_Flag', index = 'Store', aggfunc = 'mean')
#let 0 be non-holiday and 1 be holiday in the holiday flag column
holiday_effect.columns = ['Non-Holiday', 'Holiday']
holiday_effect.head()

,Non-Holiday,Holiday
Store,,
1,1.546957e+06,1665747.656
2,1.914209e+06,2079266.900
3,4.000648e+05,437811.050
4,2.083556e+06,2243102.624
5,3.148923e+05,359501.607


## Seasonal & Monthly Trends

In [3]:
# convert date to datetime type
walmart['Date'] = pd.to_datetime(walmart['Date'], format='%d-%m-%Y')
walmart['Year'] = walmart['Date'].dt.year

# How do sales fluctuate throughout the year?
sales_by_year = walmart.groupby('Year', as_index=False)['Weekly_Sales'].sum()
print(sales_by_year)


   Year  Weekly_Sales
0  2010  2.288886e+09
1  2011  2.448200e+09
2  2012  2.000133e+09


In [ ]:
# extract months from the date 
walmart['Month'] = walmart['Date'].dt.month_name()

In [ ]:
# create avgerage sales columns
walmart['avg_sale'] = walmart['Weekly_Sales'].mean()

In [11]:
# Calculate average sales per month
avg_monthly_sales = walmart.groupby('Month')['Weekly_Sales'].mean()

# Find highest and lowest average sales months
highest_month = avg_monthly_sales.idxmax()
lowest_month = avg_monthly_sales.idxmin()

print(f"Highest Average Sales Month: {highest_month}")
print(f"Lowest Average Sales Month: {lowest_month}")
# avg_monthly_sales


Highest Average Sales Month: December
Lowest Average Sales Month: January


In [12]:
# Calculate average sales per month
avg_monthly_sales = walmart.groupby('Month')['Weekly_Sales'].mean().reset_index()

# Identify the highest and lowest average sales months
highest = avg_monthly_sales.loc[avg_monthly_sales['Weekly_Sales'].idxmax()]
lowest = avg_monthly_sales.loc[avg_monthly_sales['Weekly_Sales'].idxmin()]

print("Highest Average Sales Month:")
print(highest)
print("\nLowest Average Sales Month:")
print(lowest)


Highest Average Sales Month:
Month                 December
Weekly_Sales    1281863.633778
Name: 2, dtype: object

Lowest Average Sales Month:
Month                 January
Weekly_Sales    923884.551361
Name: 4, dtype: object


In [13]:
walmart.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,avg_sale
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2010,February,1.046965e+06
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,2010,February,1.046965e+06
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010,February,1.046965e+06
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010,February,1.046965e+06
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,2010,March,1.046965e+06


In [14]:
walmart.drop(columns="avg_sale")

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2010,February
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,2010,February
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010,February
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010,February
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,2010,March
...,...,...,...,...,...,...,...,...,...,...
6430,45,2012-09-28,713173.95,0,64.88,3.997,192.013558,8.684,2012,September
6431,45,2012-10-05,733455.07,0,64.89,3.985,192.170412,8.667,2012,October
6432,45,2012-10-12,734464.36,0,54.47,4.000,192.327265,8.667,2012,October
6433,45,2012-10-19,718125.53,0,56.47,3.969,192.330854,8.667,2012,October
